In [ ]:
!pip install --upgrade --force-reinstall numpy gensim spacy adjustText umap-learn pymorphy3

In [ ]:
import os
import requests
import numpy as np
from tqdm import tqdm
from gensim.models import KeyedVectors
import zipfile
import gzip
import shutil
from pymorphy3 import MorphAnalyzer
from collections import defaultdict

class RusVectoresAnalyzer:
    def __init__(self):
        self.model = None
        self.morph = MorphAnalyzer()
        self.pos_cache = defaultdict(dict)

    def _validate_model_file(self, file_path):
        """Проверка целостности файла модели"""
        try:
            print(f"\nПроверка файла модели: {file_path}")
            print(f"Размер файла: {os.path.getsize(file_path) / (1024*1024):.2f} MB")

            if file_path.endswith('.gz'):
                print("Проверка gzip архива...")
                with gzip.open(file_path, 'rb') as f:
                    f.read(100)
                print("Gzip архив валиден")

            elif file_path.endswith('.zip'):
                print("Проверка zip архива...")
                with zipfile.ZipFile(file_path) as z:
                    bad_file = z.testzip()
                    if bad_file:
                        raise ValueError(f"Поврежденный файл в архиве: {bad_file}")
                print("Zip архив валиден")

            return True
        except Exception as e:
            print(f"Ошибка проверки файла: {e}")
            return False

    def download_model(self):
        """Улучшенная загрузка модели с проверками"""
        MODEL_URL = "https://vectors.nlpl.eu/repository/20/220.zip"
        ZIP_PATH = "rusvectores_model.zip"
        MODEL_FILE = None

        # Шаг 1: Загрузка архива
        if not os.path.exists(ZIP_PATH):
            print(f"\nЗагрузка модели с {MODEL_URL}...")
            try:
                response = requests.get(MODEL_URL, stream=True)
                response.raise_for_status()

                total_size = int(response.headers.get('content-length', 0))
                if total_size < 1024*1024:
                    raise ValueError(f"Слишком маленький файл ({total_size} bytes)")

                with open(ZIP_PATH, 'wb') as f, tqdm(
                    desc="Прогресс",
                    total=total_size,
                    unit='iB',
                    unit_scale=True,
                ) as bar:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            bar.update(len(chunk))

                if not self._validate_model_file(ZIP_PATH):
                    raise ValueError("Загруженный архив поврежден")

            except Exception as e:
                print(f"Ошибка загрузки: {e}")
                if os.path.exists(ZIP_PATH):
                    os.remove(ZIP_PATH)
                raise

        # Шаг 2: Распаковка архива
        print("\nПоиск файлов модели в архиве...")
        try:
            with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
                file_list = zip_ref.namelist()
                print("Содержимое архива:")
                for f in file_list:
                    print(f" - {f} (размер: {zip_ref.getinfo(f).file_size / (1024*1024):.2f} MB)")

                # Ищем файлы модели (поддерживаемые расширения)
                valid_extensions = ['.bin', '.vec', '.txt', '.model', '.gz']
                model_files = [f for f in file_list if any(f.endswith(ext) for ext in valid_extensions)]

                if not model_files:
                    raise FileNotFoundError("Не найдены файлы модели в архиве")

                # Выбираем первый подходящий файл
                MODEL_FILE = model_files[0]
                print(f"\nИзвлечение файла модели: {MODEL_FILE}")
                zip_ref.extract(MODEL_FILE)

                # Проверка извлеченного файла
                if not os.path.exists(MODEL_FILE):
                    raise FileNotFoundError(f"Файл {MODEL_FILE} не был извлечен")

                print(f"Успешно извлечен: {MODEL_FILE} (размер: {os.path.getsize(MODEL_FILE) / (1024*1024):.2f} MB)")

                # Распаковка gzip если нужно
                if MODEL_FILE.endswith('.gz'):
                    uncompressed_file = MODEL_FILE[:-3]
                    print(f"\nРаспаковка gzip: {MODEL_FILE} -> {uncompressed_file}")

                    with gzip.open(MODEL_FILE, 'rb') as f_in:
                        with open(uncompressed_file, 'wb') as f_out:
                            shutil.copyfileobj(f_in, f_out)

                    if not os.path.exists(uncompressed_file):
                        raise ValueError("Ошибка распаковки gzip")

                    MODEL_FILE = uncompressed_file
                    print(f"Успешно распакован: {MODEL_FILE} (размер: {os.path.getsize(MODEL_FILE) / (1024*1024):.2f} MB)")

        except Exception as e:
            print(f"Ошибка распаковки: {e}")
            raise

        # Шаг 3: Загрузка модели с проверкой
        print("\nПопытка загрузки модели...")
        try:
            # Определяем формат файла
            if MODEL_FILE.endswith('.bin'):
                print("Загрузка в бинарном формате...")
                self.model = KeyedVectors.load_word2vec_format(MODEL_FILE, binary=True)
            elif MODEL_FILE.endswith('.vec') or MODEL_FILE.endswith('.txt'):
                print("Загрузка в текстовом формате...")
                self.model = KeyedVectors.load_word2vec_format(MODEL_FILE, binary=False)
            else:
                print("Попытка загрузки через gensim...")
                self.model = KeyedVectors.load(MODEL_FILE)

            # Проверка загруженной модели
            if not hasattr(self.model, 'vectors'):
                raise AttributeError("Модель не содержит векторов")

            print(f"\nМодель успешно загружена!")
            print(f"Количество слов: {len(self.model.index_to_key)}")
            print(f"Размерность векторов: {self.model.vector_size}")
            print("Примеры слов:", self.model.index_to_key[:5])

        except Exception as e:
            print(f"\nОшибка загрузки модели: {e}")
            print("\nДиагностика:")
            # Попробуем прочитать первые строки файла
            try:
                with open(MODEL_FILE, 'rb') as f:
                    head = f.read(200)
                    print("Начало файла (hex):", head[:100].hex())
                    try:
                        print("Начало файла (text):", head[:100].decode('utf-8', errors='replace'))
                    except:
                        pass
            except Exception as e:
                print(f"Не удалось прочитать файл: {e}")

            raise

    def _is_noun(self, word):
        """Проверяет, является ли слово существительным и возвращает нормальную форму"""
        if word in self.pos_cache:
            return self.pos_cache[word]['is_noun'], self.pos_cache[word]['normal_form']

        parsed = self.morph.parse(word)
        if not parsed:
            self.pos_cache[word] = {'is_noun': False, 'normal_form': None}
            return False, None

        # Ищем разбор с максимальной вероятностью и существительным
        best_parse = max(parsed, key=lambda p: p.score)
        is_noun = 'NOUN' in best_parse.tag
        normal_form = best_parse.normal_form if is_noun else None

        self.pos_cache[word] = {'is_noun': is_noun, 'normal_form': normal_form}
        return is_noun, normal_form

    def get_nouns(self):
        """Получение существительных с проверкой через pymorphy3 и нормализацией"""
        if self.model is None:
            raise ValueError("Модель не загружена")

        print("\nПоиск и проверка существительных...")
        unique_lemmas = set()
        vectors = []
        nouns = []
        skipped = 0
        form_errors = 0

        for word in tqdm(self.model.index_to_key, desc="Обработка"):
            if len(unique_lemmas) >= 10000:
                break

            if not word.endswith('_NOUN'):
                skipped += 1
                continue

            try:
                clean_word = word[:-5]

                # Морфологический анализ
                is_noun, normal_form = self._is_noun(clean_word)
                if not is_noun or not normal_form:
                    skipped += 1
                    continue

                # Дополнительная проверка нормальной формы
                if not self.morph.parse(normal_form)[0].tag.POS == 'NOUN':
                    form_errors += 1
                    continue

                # Проверка уникальности
                if normal_form in unique_lemmas:
                    skipped += 1
                    continue

                # Получаем и проверяем вектор
                vector = self.model[word]
                if np.isnan(vector).any() or np.isinf(vector).any():
                    raise ValueError("Невалидный вектор")

                # Сохраняем данные
                unique_lemmas.add(normal_form)
                nouns.append(normal_form)
                vectors.append(vector)

            except Exception as e:
                skipped += 1
                continue

        # Проверка количества найденных лемм
        if len(unique_lemmas) < 10000:
            raise ValueError(f"Недостаточно уникальных существительных. Найдено: {len(unique_lemmas)}")

        print(f"\nРезультаты:")
        print(f"Уникальных существительных: {len(unique_lemmas)}")
        print(f"Всего обработано слов: {len(vectors)}")
        print(f"Пропущено: {skipped}")
        print(f"Ошибки нормализации: {form_errors}")

        return np.array(vectors), nouns

if __name__ == "__main__":
    try:
        analyzer = RusVectoresAnalyzer()
        analyzer.download_model()

        selected_vectors, selected_nouns = analyzer.get_nouns()
        print("\nПервые 10 лемм:", selected_nouns[:10])
        print("Размерность векторов:", selected_vectors.shape)
        print("Уникальных лемм:", len(set(selected_nouns)))

    except Exception as e:
        print(f"\nОшибка: {e}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.manifold import TSNE
from tqdm import tqdm
from collections import defaultdict
import plotly.express as px
from adjustText import adjust_text
from gensim.models import KeyedVectors
from scipy.stats import entropy
from sklearn.preprocessing import normalize

class Word2VecSemanticAnalyzer:
    def __init__(self, vectors, words, random_state=42):
        """
        Инициализация анализатора с векторами слов и их текстовыми представлениями

        Параметры:
        vectors - numpy array с векторными представлениями слов
        words - список слов, соответствующих векторам
        random_state - seed для воспроизводимости
        """
        self.vectors = vectors
        self.words = words
        self.random_state = random_state
        self.reduced_vectors = None
        self.gmm = None
        self.optimal_k = None
        self.sub_level_gmms = {}
        self.stationary_indices = defaultdict(list)
        self.cluster_metrics = []
        self.word_to_idx = {word: idx for idx, word in enumerate(words)}

    def reduce_dimensions(self, n_components=200, variance_threshold=0.01):
        """Снижение размерности с PCA и автоматическим выбором компонент"""
        pca = PCA(n_components=min(n_components, self.vectors.shape[1]),
                 random_state=self.random_state)
        self.reduced_vectors = pca.fit_transform(self.vectors)

        # Автоматический выбор числа компонент по порогу дисперсии
        cum_var = np.cumsum(pca.explained_variance_ratio_)
        optimal_components = np.where(pca.explained_variance_ratio_ > variance_threshold)[0].size
        optimal_components = max(100, optimal_components)  # Не меньше 100 компонент

        plt.figure(figsize=(12, 6))
        plt.subplot(121)
        plt.plot(pca.explained_variance_ratio_, 'o-')
        plt.axvline(optimal_components, color='r', linestyle='--')
        plt.title('Объясненная дисперсия по компонентам')
        plt.xlabel('Номер компоненты')
        plt.ylabel('Доля объясненной дисперсии')

        plt.subplot(122)
        plt.plot(cum_var, 'o-')
        plt.axvline(optimal_components, color='r', linestyle='--')
        plt.axhline(cum_var[optimal_components], color='g', linestyle='--')
        plt.title('Накопленная объясненная дисперсия')
        plt.xlabel('Число компонент')
        plt.ylabel('Накопленная дисперсия')

        plt.tight_layout()
        plt.show()

        # Если нашли лучшее число компонент - пересчитываем
        if optimal_components != n_components:
            print(f"Оптимальное число компонент: {optimal_components}")
            pca = PCA(n_components=optimal_components, random_state=self.random_state)
            self.reduced_vectors = pca.fit_transform(self.vectors)

        return self.reduced_vectors

    def find_optimal_clusters(self, max_k=50, min_k=10, step=5, n_init=5):
        """Определение оптимального числа кластеров с улучшенной проверкой данных"""
        cluster_range = range(min_k, max_k + 1, step)

        # Проверка входных данных
        if self.reduced_vectors is None:
            raise ValueError("Сначала выполните снижение размерности!")

        if np.isnan(self.reduced_vectors).any() or np.isinf(self.reduced_vectors).any():
            raise ValueError("Обнаружены NaN/Inf значения в данных. Проверьте векторы.")

        metrics = {
            'silhouette': [],
            'davies_bouldin': [],
            'combo': [],
            'bic': [],
            'aic': []
        }

        best_metrics = {
            'silhouette': -np.inf,
            'davies_bouldin': np.inf,
            'combo': -np.inf,
            'k': None
        }

        for k in tqdm(cluster_range, desc="Кластеризация"):
            try:
                gmm = GaussianMixture(
                    n_components=k,
                    covariance_type='diag',
                    random_state=self.random_state,
                    n_init=n_init,
                    max_iter=300,
                    tol=1e-4,
                    reg_covar=1e-3
                )
                gmm.fit(self.reduced_vectors)

                # Проверка сходимости
                if not gmm.converged_:
                    raise ValueError("GMM не сошелся")

                labels = gmm.predict(self.reduced_vectors)

                # Вычисление метрик
                sil_score = silhouette_score(self.reduced_vectors, labels)
                db_score = davies_bouldin_score(self.reduced_vectors, labels)
                combo_score = 0.7*sil_score - 0.3*db_score

                # Обновление лучшего результата
                if combo_score > best_metrics['combo']:
                    best_metrics.update({
                        'silhouette': sil_score,
                        'davies_bouldin': db_score,
                        'combo': combo_score,
                        'k': k
                    })
                    self.optimal_k = k
                    self.gmm = gmm

                # Сохраняем метрики
                metrics['silhouette'].append(sil_score)
                metrics['davies_bouldin'].append(db_score)
                metrics['combo'].append(combo_score)
                metrics['bic'].append(gmm.bic(self.reduced_vectors))
                metrics['aic'].append(gmm.aic(self.reduced_vectors))

            except Exception as e:
                print(f"Ошибка для k={k}: {str(e)}")
                for name in metrics.keys():
                    metrics[name].append(None)


        # Визуализация метрик
        self._plot_cluster_metrics(cluster_range, metrics)

        print(f"\nОптимальное число кластеров: {self.optimal_k}")
        print(f"Silhouette: {best_metrics['silhouette']:.3f}")
        print(f"Davies-Bouldin: {best_metrics['davies_bouldin']:.3f}")

        return self.optimal_k

    def _plot_cluster_metrics(self, cluster_range, metrics):
        """Улучшенная визуализация с проверкой значений"""
        plt.figure(figsize=(18, 10))

        # Проверка данных для графиков
        valid_metrics = {
            name: [(x is not None) for x in vals]
            for name, vals in metrics.items()
        }

        # Silhouette Score
        plt.subplot(2, 2, 1)
        if any(valid_metrics['silhouette']):
            plt.plot(cluster_range, metrics['silhouette'], 'o-')
            if self.optimal_k is not None:
                plt.axvline(self.optimal_k, color='r', linestyle='--')
        plt.title('Silhouette Score')

        # Davies-Bouldin Index
        plt.subplot(2, 2, 2)
        if any(valid_metrics['davies_bouldin']):
            plt.plot(cluster_range, metrics['davies_bouldin'], 'o-', color='orange')
            if self.optimal_k is not None:
                plt.axvline(self.optimal_k, color='r', linestyle='--')
        plt.title('Davies-Bouldin Index')

        # Комбинированная метрика
        plt.subplot(2, 2, 3)
        if any(valid_metrics['combo']):
            plt.plot(cluster_range, metrics['combo'], 'o-', color='green')
            if self.optimal_k is not None:
                plt.axvline(self.optimal_k, color='r', linestyle='--')
        plt.title('Комбинированная метрика')

        # Информационные критерии
        plt.subplot(2, 2, 4)
        if any(valid_metrics['bic']) and any(valid_metrics['aic']):
            plt.plot(cluster_range, metrics['bic'], 'o-', color='purple', label='BIC')
            plt.plot(cluster_range, metrics['aic'], 'o-', color='brown', label='AIC')
            if self.optimal_k is not None:
                plt.axvline(self.optimal_k, color='r', linestyle='--')
            plt.legend()

        plt.tight_layout()
        plt.show()

    def visualize_clusters(self, max_points=1000, perplexity=30):
        """Улучшенная визуализация кластеров"""
        if self.gmm is None:
            raise ValueError("Сначала выполните кластеризацию!")

        labels = self.gmm.predict(self.reduced_vectors)
        probs = self.gmm.predict_proba(self.reduced_vectors)
        uncertainties = 1 - np.max(probs, axis=1)

        # Выбор подмножества для визуализации
        if len(self.reduced_vectors) > max_points:
            # Выбираем точки с наибольшей уверенностью кластеризации
            confident_indices = np.argsort(-uncertainties)[:max_points]
            sample_vectors = self.reduced_vectors[confident_indices]
            sample_words = [self.words[i] for i in confident_indices]
            sample_labels = labels[confident_indices]
            sample_uncertainties = uncertainties[confident_indices]
        else:
            sample_vectors = self.reduced_vectors
            sample_words = self.words
            sample_labels = labels
            sample_uncertainties = uncertainties

        # t-SNE проекция с адаптивным perplexity
        perplexity = min(perplexity, len(sample_vectors) // 3 - 1)
        tsne = TSNE(
            n_components=2,
            random_state=self.random_state,
            perplexity=perplexity,
            init='pca',
            learning_rate='auto'
        )
        tsne_vectors = tsne.fit_transform(sample_vectors)

        # Интерактивная визуализация с Plotly (с неопределенностью)
        fig = px.scatter(
            x=tsne_vectors[:, 0], y=tsne_vectors[:, 1],
            color=sample_labels.astype(str),
            size=1-sample_uncertainties,
            hover_name=sample_words,
            title=f"Семантические кластеры (k={self.optimal_k})<br>Размер точки отражает уверенность кластеризации",
            width=1000, height=800,
            color_discrete_sequence=px.colors.qualitative.Alphabet
        )
        fig.update_traces(
            marker=dict(line=dict(width=0.5, color='DarkSlateGrey')),
            selector=dict(mode='markers')
        )
        fig.show()

        # Статическая визуализация с подписями
        self._plot_static_clusters(tsne_vectors, sample_labels, sample_words, sample_uncertainties)

    def _plot_static_clusters(self, tsne_vectors, labels, words, uncertainties):
        """Улучшенная статическая визуализация с подписями"""
        plt.figure(figsize=(20, 15))

        # Цвета по кластерам, размер по уверенности
        scatter = plt.scatter(
            tsne_vectors[:, 0], tsne_vectors[:, 1],
            c=labels, cmap='tab20',
            s=100*(1-uncertainties),
            alpha=0.7,
            edgecolor='k',
            linewidths=0.5
        )

        # Добавляем подписи для части точек (выбираем наиболее типичные)
        texts = []
        cluster_centers = {}

        # Находим центры кластеров в t-SNE пространстве
        for cluster in np.unique(labels):
            mask = labels == cluster
            center = np.median(tsne_vectors[mask], axis=0)
            cluster_centers[cluster] = center

            # Выбираем ближайшие точки к центру
            distances = np.linalg.norm(tsne_vectors[mask] - center, axis=1)
            closest_idx = np.argsort(distances)[:5]  # 5 ближайших слов
            closest_indices = np.where(mask)[0][closest_idx]

            for idx in closest_indices:
                texts.append(plt.text(
                    tsne_vectors[idx, 0], tsne_vectors[idx, 1],
                    words[idx], fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1)
                ))

        adjust_text(
            texts,
            arrowprops=dict(arrowstyle='-', color='gray', lw=0.5),
            expand_points=(1.5, 1.5),
            expand_text=(1.2, 1.2)
        )
        plt.title(f"Семантические кластеры (k={self.optimal_k})\nРазмер точки отражает уверенность кластеризации", pad=20)
        plt.axis('off')

        # Добавляем легенду с номерами кластеров
        handles, _ = scatter.legend_elements()
        plt.legend(
            handles,
            [f"Кластер {i}" for i in np.unique(labels)],
            title="Кластеры",
            bbox_to_anchor=(1, 1),
            loc='upper left'
        )

        plt.tight_layout()
        plt.show()

    def hierarchical_clustering(self, max_subclusters=10, min_subclusters=3,
                           max_points=200, min_cluster_size=20):
        cluster_sizes = np.bincount(self.gmm.predict(self.reduced_vectors))

        for cluster_idx in range(self.optimal_k):
            size = cluster_sizes[cluster_idx]
            if size < min_cluster_size:
                print(f"Кластер {cluster_idx} слишком мал ({size} точек), пропускаем")
                continue

            cluster_mask = self.gmm.predict(self.reduced_vectors) == cluster_idx
            cluster_points = self.reduced_vectors[cluster_mask]

            if np.isnan(cluster_points).any():
                print(f"Кластер {cluster_idx} содержит NaN значения, пропускаем")
                continue

            self._process_single_cluster(
                cluster_idx,
                max_subclusters,
                min_subclusters,
                max_points
            )

    def _process_single_cluster(self, cluster_idx, max_subclusters, min_subclusters, max_points):
        """Обработка одного кластера верхнего уровня с улучшенным анализом"""
        cluster_mask = self.gmm.predict(self.reduced_vectors) == cluster_idx
        cluster_points = self.reduced_vectors[cluster_mask]
        cluster_words = [self.words[i] for i in np.where(cluster_mask)[0]]

        print(f"\n{'='*50}")
        print(f"Анализ кластера {cluster_idx} ({len(cluster_points)} слов)")
        print(f"Примеры слов: {', '.join(np.random.choice(cluster_words, min(10, len(cluster_words))))}")
        print("="*50)

        if len(cluster_points) < min_subclusters:
            print("Слишком мало точек для подкластеризации")
            return

        # Определение оптимального числа подкластеров
        optimal_sub_k, metrics = self._find_optimal_subclusters(
            cluster_points, max_subclusters, min_subclusters
        )

        if optimal_sub_k < 2:
            print("Не удалось найти значимые подкластеры")
            return

        # Кластеризация и визуализация
        gmm_sub, sub_labels, original_indices = self._perform_subclustering(
        cluster_points, cluster_words, cluster_idx, optimal_sub_k, max_points
        )

        self._store_subcluster_results(cluster_idx, cluster_mask, sub_labels, gmm_sub, original_indices)

        # Визуализация метрик
        self._plot_subcluster_metrics(metrics, optimal_sub_k)

        # Анализ семантической когерентности подкластеров
        self._analyze_subclusters_semantics(cluster_words, sub_labels)

    def _find_optimal_subclusters(self, points, max_k, min_k):
        """Улучшенный метод определения оптимального числа подкластеров"""
        sub_range = range(min_k, max_k+1)
        metrics = {
            'k': [],
            'silhouette': [],
            'davies_bouldin': [],
            'bic': [],
            'combo': [],
            'gap': []
        }

        best_k = min_k
        best_combo = -np.inf
        prev_sil = -1

        for k in sub_range:
            metrics['k'].append(k)
            if len(points) < k*2:
                print(f"Слишком мало точек ({len(points)}) для k={k}")
                for m in ['silhouette', 'davies_bouldin', 'bic', 'combo', 'gap']:
                    metrics[m].append(None)
                continue

            try:
                gmm = GaussianMixture(
                    n_components=k,
                    covariance_type='diag',
                    random_state=self.random_state,
                    n_init=5,
                    reg_covar=1e-1,
                    tol=1e-4,
                    max_iter=500
                )
                gmm.fit(points)
                labels = gmm.predict(points)

                unique_labels = np.unique(labels)
                if len(unique_labels) < k:
                    raise ValueError(f"Обнаружены пустые кластеры для k={k}")

                sil_score = silhouette_score(points, labels) if k > 1 else 0
                db_score = davies_bouldin_score(points, labels) if k > 1 else np.inf
                bic = gmm.bic(points)
                combo_score = 0.7*sil_score - 0.3*db_score

                try:
                    ref_disps = []
                    for _ in range(3):
                        # Генерация синтетических данных
                        random_data = np.random.rand(*points.shape) * (points.max(0)-points.min(0)) + points.min(0)

                        # Фильтрация NaN/Inf
                        random_data = np.nan_to_num(random_data, nan=0.0, posinf=1e6, neginf=-1e6)

                        # Вычисление логарифма с защитой
                        gmm_ref = GaussianMixture(
                            n_components=k,
                            random_state=self.random_state,
                            reg_covar=1e-1
                        )
                        gmm_ref.fit(random_data)
                        log_prob = gmm_ref.score(random_data)
                        ref_disps.append(log_prob if log_prob > -np.inf else 0)

                    current_score = gmm.score(points)
                    gap = np.mean(ref_disps) - (current_score if current_score > -np.inf else 0)
                except Exception as e:
                    print(f"Ошибка при вычислении Gap Statistic: {str(e)}")
                    gap = 0

                metrics['silhouette'].append(sil_score)
                metrics['davies_bouldin'].append(db_score)
                metrics['bic'].append(bic)
                metrics['combo'].append(combo_score)
                metrics['gap'].append(gap)

                # Обновление лучшего k
                if combo_score > best_combo and gap > 0:
                    best_combo = combo_score
                    best_k = k

                prev_sil = sil_score

            except Exception as e:
                print(f"Ошибка для k={k}: {str(e)}")
                for m in ['silhouette', 'davies_bouldin', 'bic', 'combo', 'gap']:
                    metrics[m].append(None)

        return best_k, metrics

    def _perform_subclustering(self, points, words, cluster_idx, optimal_k, max_points=500):
        """Улучшенная визуализация подкластеров с подписями всех точек"""
        gmm_sub = GaussianMixture(
                n_components=optimal_k,
                covariance_type='diag',
                random_state=self.random_state,
                n_init=5,
                reg_covar=1e-1,
                tol=1e-4,
                max_iter=500
            )
        sub_labels = gmm_sub.fit_predict(points)

        # Проверка на NaN в метках
        if np.isnan(sub_labels).any():
            raise ValueError("Обнаружены NaN в метках кластеров")

        # Добавляем расчет неопределенности
        probs = gmm_sub.predict_proba(points)
        uncertainties = 1 - np.max(probs, axis=1)

        # Сохраняем оригинальные индексы до выборки
        original_indices = np.arange(len(points))

        # Ограничиваем количество точек для визуализации
        if len(points) > max_points:
            indices = np.random.choice(len(points), max_points, replace=False)
            points = points[indices]
            words = [words[i] for i in indices]
            sub_labels = sub_labels[indices]
            uncertainties = uncertainties[indices]
            original_indices = original_indices[indices]

        # t-SNE проекция
        perplexity = min(50, len(points)-1)
        tsne = TSNE(
            n_components=2,
            perplexity=perplexity,
            random_state=self.random_state,
            init='pca',
            learning_rate='auto'
        )
        embedded = tsne.fit_transform(points)

        # Создаем фигуру
        plt.figure(figsize=(20, 15))

        # Размер точек зависит от уверенности кластеризации (1 - uncertainty)
        sizes = 50 + 150 * (1 - uncertainties)

        # Цвета для подкластеров
        colors = plt.cm.tab20(np.linspace(0, 1, optimal_k))

        # Рисуем точки с подписями
        texts = []
        for i in range(len(points)):
            plt.scatter(
                embedded[i, 0], embedded[i, 1],
                color=colors[sub_labels[i]],
                s=sizes[i],
                alpha=0.7,
                edgecolor='k',
                linewidths=0.3
            )
            texts.append(plt.text(
                embedded[i, 0], embedded[i, 1],
                words[i],
                fontsize=8,
                color='black',
                ha='center', va='center'
            ))

        # Настройка легенды
        legend_elements = [
            plt.Line2D([0], [0], marker='o', color='w',
                      markerfacecolor=colors[i], markersize=10,
                      label=f'Подкластер {i}')
            for i in range(optimal_k)
        ]

        plt.legend(
            handles=legend_elements,
            title='Подкластеры',
            bbox_to_anchor=(1.05, 1),
            loc='upper left'
        )

        # Настройка заголовка
        plt.title(
            f"Разбиение кластера {cluster_idx} на {optimal_k} подкластеров\n"
            f"Размер точки отражает уверенность кластеризации\n"
            f"Всего точек: {len(points)}",
            pad=20
        )

        # Автоматическая регулировка подписей
        adjust_text(
            texts,
            arrowprops=dict(arrowstyle='-', color='gray', lw=0.5),
            expand_points=(1.2, 1.2),
            expand_text=(1.1, 1.1),
            force_text=(0.5, 0.5),
            force_points=(0.8, 0.8),
            lim=1000
        )

        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

        return gmm_sub, sub_labels, original_indices

    def _plot_subcluster_metrics(self, metrics, optimal_k):
        """Исправленная визуализация метрик подкластеров"""
        plt.figure(figsize=(15,10))

        # Фильтруем невалидные значения
        valid = ~np.isnan(metrics['silhouette'])
        k_values = np.array(metrics['k'])[valid]

        plt.subplot(2,2,1)
        plt.plot(k_values, np.array(metrics['silhouette'])[valid], 'o-')
        plt.axvline(optimal_k, color='r', linestyle='--')
        plt.title('Silhouette Score')

        plt.subplot(2,2,2)
        plt.plot(k_values, np.array(metrics['davies_bouldin'])[valid], 'o-', color='orange')
        plt.axvline(optimal_k, color='r', linestyle='--')
        plt.title('Davies-Bouldin Index')

        plt.subplot(2,2,3)
        plt.plot(k_values, np.array(metrics['combo'])[valid], 'o-', color='green')
        plt.axvline(optimal_k, color='r', linestyle='--')
        plt.title('Комбинированная метрика')

        plt.subplot(2,2,4)
        plt.plot(k_values, np.array(metrics['gap'])[valid], 'o-', color='purple')
        plt.axvline(optimal_k, color='r', linestyle='--')
        plt.title('Gap Statistic')

        plt.tight_layout()
        plt.show()

    def _analyze_subclusters_semantics(self, words, labels):
        """Анализ семантической когерентности подкластеров"""
        unique_labels = np.unique(labels)
        print("\nСемантический анализ подкластеров:")

        for sub_id in unique_labels:
            cluster_words = [w for w, l in zip(words, labels) if l == sub_id]
            print(f"\nПодкластер {sub_id} ({len(cluster_words)} слов):")
            print(", ".join(cluster_words[:15]) + ("..." if len(cluster_words) > 15 else ""))

    def _store_subcluster_results(self, cluster_idx, mask, sub_labels, gmm_sub, original_indices):
        """Сохранение результатов подкластеризации"""
        self.sub_level_gmms[cluster_idx] = gmm_sub
        # Получаем глобальные индексы для всего кластера
        cluster_global_indices = np.where(mask)[0]

        for sub_id in range(gmm_sub.n_components):
            # Маска для текущего подкластера в ограниченной выборке
            sub_mask = sub_labels == sub_id
            # Преобразуем в глобальные индексы
            global_indices = cluster_global_indices[original_indices[sub_mask]]
            for idx in global_indices:
                self.stationary_indices[idx].append((cluster_idx, sub_id))

    def get_cluster_words(self, top_n=10, sort_clusters=True):
        """Получение топ-слов для каждого кластера с учетом вероятностей"""
        cluster_words = defaultdict(list)
        probs = self.gmm.predict_proba(self.reduced_vectors)

        for word, label, prob in zip(self.words, self.gmm.predict(self.reduced_vectors), probs):
            cluster_words[label].append((word, np.max(prob)))

        # Сортируем слова по вероятности принадлежности к кластеру
        result = {}
        for k in cluster_words:
            sorted_words = sorted(cluster_words[k], key=lambda x: -x[1])
            result[k] = [w[0] for w in sorted_words[:top_n]]

        return dict(sorted(result.items())) if sort_clusters else result

    def print_cluster_words(self, top_n=15):
        """Вывод топ-слов с информацией о размере кластера"""
        cluster_words = self.get_cluster_words(top_n=top_n)
        labels = self.gmm.predict(self.reduced_vectors)
        cluster_sizes = np.bincount(labels)

        print("\nТоп слов по кластерам (размер | средняя вероятность):")
        for cluster, words in cluster_words.items():
            probas = self.gmm.predict_proba(self.reduced_vectors)[labels == cluster]
            avg_prob = np.mean(np.max(probas, axis=1))

            print(f"\nКластер {cluster} [размер: {cluster_sizes[cluster]} | prob: {avg_prob:.2f}]:")
            print(", ".join(words[:top_n]))

    def analyze_cluster_quality(self, sample_size=1000):
        """Улучшенный анализ семантической когерентности кластеров"""
        try:
            # Создаем временную модель для вычисления схожести
            temp_model = KeyedVectors(vector_size=self.vectors.shape[1])
            temp_model.add_vectors(self.words, self.vectors)

            cluster_coherence = {}
            cluster_diversity = {}
            labels = self.gmm.predict(self.reduced_vectors)
            cluster_sizes = np.bincount(labels)  # Вычисляем размеры кластеров

            for cluster in np.unique(labels):
                mask = labels == cluster
                cluster_words = [self.words[i] for i in np.where(mask)[0]]

                if len(cluster_words) < 2:
                    cluster_coherence[cluster] = 0
                    cluster_diversity[cluster] = 0
                    continue

                # Вычисляем когерентность (средняя попарная схожесть)
                similarities = []
                for i in range(min(len(cluster_words), sample_size)):
                    for j in range(i+1, min(len(cluster_words), sample_size)):
                        try:
                            sim = temp_model.similarity(cluster_words[i], cluster_words[j])
                            similarities.append(sim)
                        except:
                            continue

                cluster_coherence[cluster] = np.mean(similarities) if similarities else 0

                # Вычисляем диверсификацию (энтропия по топ-ближайшим словам)
                diversities = []
                for word in cluster_words[:sample_size]:
                    try:
                        similars = temp_model.most_similar(word, topn=10)
                        similar_words = [w for w, _ in similars]
                        word_counts = [cluster_words.count(w) for w in similar_words]
                        word_probs = np.array(word_counts) / len(cluster_words)
                        word_probs = word_probs[word_probs > 0]
                        diversities.append(entropy(word_probs))
                    except:
                        continue

                cluster_diversity[cluster] = np.mean(diversities) if diversities else 0

            self._plot_cluster_quality(cluster_coherence, cluster_diversity, cluster_sizes)

            return {
                'coherence': cluster_coherence,
                'diversity': cluster_diversity,
                'cluster_sizes': cluster_sizes
            }

        except Exception as e:
            print(f"Ошибка анализа когерентности: {e}")
            return None

    def _plot_cluster_quality(self, coherence, diversity, cluster_sizes):
        """Визуализация качества кластеров с размерами"""
        clusters = sorted(coherence.keys())
        coh_values = [coherence[c] for c in clusters]
        div_values = [diversity[c] for c in clusters]
        size_values = [cluster_sizes[c] for c in clusters]

        plt.figure(figsize=(18, 6))

        # График когерентности с размерами
        plt.subplot(131)
        bars = plt.bar(clusters, coh_values, alpha=0.7)
        plt.title('Семантическая когерентность\n(высота - метрика, цвет - размер)')
        plt.xlabel('Номер кластера')
        plt.ylabel('Средняя попарная схожесть')

        # Добавляем аннотации с размерами и цветовую индикацию
        max_size = max(size_values)
        for bar, size in zip(bars, size_values):
            bar.set_color(plt.cm.viridis(size / max_size))
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                    f'{size}', ha='center', va='bottom')

        # График диверсификации
        plt.subplot(132)
        plt.bar(clusters, div_values, color='orange', alpha=0.7)
        plt.title('Семантическая диверсификация')
        plt.xlabel('Номер кластера')
        plt.ylabel('Энтропия ближайших слов')

        # График размеров кластеров
        plt.subplot(133)
        plt.bar(clusters, size_values, color='green', alpha=0.7)
        plt.title('Размеры кластеров')
        plt.xlabel('Номер кластера')
        plt.ylabel('Количество слов')
        plt.yscale('log')

        plt.tight_layout()
        plt.show()

        # Совмещенный scatter plot
        plt.figure(figsize=(10, 6))
        scatter = plt.scatter(
            coh_values, div_values,
            c=size_values, cmap='viridis',
            s=np.sqrt(size_values)*10, alpha=0.7
        )
        plt.colorbar(scatter, label='Размер кластера')
        plt.title('Соотношение качества кластеров\n(Размер точки = количество слов)')
        plt.xlabel('Когерентность')
        plt.ylabel('Диверсификация')
        plt.grid(True, alpha=0.3)

        # Добавляем номера кластеров
        for i, (x, y) in enumerate(zip(coh_values, div_values)):
            plt.text(x, y, str(clusters[i]), fontsize=8,
                    ha='center', va='center', alpha=0.7)

        plt.show()

    def analyze_covariance_diff(self, cluster_idx1, cluster_idx2, top_n=5, top_k=10):
        """Улучшенный анализ различий между кластерами"""
        if cluster_idx1 not in self.sub_level_gmms or cluster_idx2 not in self.sub_level_gmms:
            raise ValueError("Указанные кластеры не существуют или не были подкластеризованы")

        gmm1 = self.sub_level_gmms[cluster_idx1]
        gmm2 = self.sub_level_gmms[cluster_idx2]

        # Получаем точки для каждого кластера
        mask1 = self.gmm.predict(self.reduced_vectors) == cluster_idx1
        mask2 = self.gmm.predict(self.reduced_vectors) == cluster_idx2
        points1 = self.reduced_vectors[mask1]
        points2 = self.reduced_vectors[mask2]
        words1 = [self.words[i] for i in np.where(mask1)[0]]
        words2 = [self.words[i] for i in np.where(mask2)[0]]

        # Нормализованные ковариации для сравнения
        cov1 = normalize(gmm1.covariances_, norm='l2')
        cov2 = normalize(gmm2.covariances_, norm='l2')

        # Вычисляем KL-дивергенцию между распределениями по измерениям
        kl_divs = []
        for dim in range(self.reduced_vectors.shape[1]):
            # Оцениваем распределения по измерениям
            hist1, _ = np.histogram(points1[:, dim], bins=20, density=True)
            hist2, _ = np.histogram(points2[:, dim], bins=20, density=True)

            # Добавляем небольшое значение чтобы избежать нулей
            hist1 = hist1 + 1e-10
            hist2 = hist2 + 1e-10
            hist1 = hist1 / hist1.sum()
            hist2 = hist2 / hist2.sum()

            kl = entropy(hist1, hist2) + entropy(hist2, hist1)
            kl_divs.append(kl)

        # Находим измерения с максимальными различиями
        top_dims = np.argsort(-np.array(kl_divs))[:top_n]

        # Собираем результаты
        results = []
        for dim in top_dims:
            # Для каждого измерения анализируем подкластеры
            for comp1 in range(gmm1.n_components):
                for comp2 in range(gmm2.n_components):
                    # Слова для компонент
                    comp1_words = [w for w, l in zip(words1, gmm1.predict(points1)) if l == comp1]
                    comp1_points = points1[gmm1.predict(points1) == comp1]
                    comp2_words = [w for w, l in zip(words2, gmm2.predict(points2)) if l == comp2]
                    comp2_points = points2[gmm2.predict(points2) == comp2]

                    if len(comp1_words) == 0 or len(comp2_words) == 0:
                        continue

                    # Топ и антонимы по измерению
                    top1 = self._get_top_words_by_dim(comp1_words, comp1_points, dim, top_k)
                    opposite1 = self._get_top_words_by_dim(comp1_words, comp1_points, dim, top_k//2, ascending=True)
                    top2 = self._get_top_words_by_dim(comp2_words, comp2_points, dim, top_k)
                    opposite2 = self._get_top_words_by_dim(comp2_words, comp2_points, dim, top_k//2, ascending=True)

                    # Средние значения по измерению
                    mean1 = np.mean(comp1_points[:, dim])
                    mean2 = np.mean(comp2_points[:, dim])

                    results.append({
                        'dimension': dim,
                        'kl_divergence': kl_divs[dim],
                        'cluster1': cluster_idx1,
                        'cluster2': cluster_idx2,
                        'component1': comp1,
                        'component2': comp2,
                        'mean1': mean1,
                        'mean2': mean2,
                        'mean_diff': abs(mean1 - mean2),
                        'cluster1_top': top1,
                        'cluster1_opposite': opposite1,
                        'cluster2_top': top2,
                        'cluster2_opposite': opposite2
                    })

        # Сортируем результаты по KL-дивергенции
        results.sort(key=lambda x: -x['kl_divergence'])
        return results[:top_n]

    def analyze_variance(self, cluster_idx, top_n=3, top_k=10):
        """Улучшенный анализ дисперсий внутри кластера"""
        if cluster_idx not in self.sub_level_gmms:
            raise ValueError("Указанный кластер не существует или не был подкластеризован")

        # Получаем точки кластера
        mask = self.gmm.predict(self.reduced_vectors) == cluster_idx
        points = self.reduced_vectors[mask]
        words = [self.words[i] for i in np.where(mask)[0]]

        # Вычисляем относительную дисперсию (по сравнению с общим распределением)
        global_var = np.var(self.reduced_vectors, axis=0)
        cluster_var = np.var(points, axis=0)
        relative_var = cluster_var / (global_var + 1e-10)

        # Находим измерения с максимальной относительной дисперсией
        top_dims = np.argsort(-relative_var)[:top_n]

        # Собираем результаты
        results = []
        for dim in top_dims:
            # Топ и антонимы по измерению
            top_words = self._get_top_words_by_dim(words, points, dim, top_k)
            opposite_words = self._get_top_words_by_dim(words, points, dim, top_k//2, ascending=True)

            # Анализ корреляций с другими измерениями
            corr_matrix = np.corrcoef(points.T)
            corr_dims = np.argsort(-np.abs(corr_matrix[dim]))[1:top_n+1]

            results.append({
                'dimension': dim,
                'variance': cluster_var[dim],
                'relative_variance': relative_var[dim],
                'cluster': cluster_idx,
                'top_words': top_words,
                'opposite_words': opposite_words,
                'correlated_dims': corr_dims.tolist(),
                'correlation_values': corr_matrix[dim, corr_dims].tolist()
            })

        return results

    def _get_top_words_by_dim(self, words, vectors, dim, n_words, ascending=False):
        """Вспомогательный метод для получения топ-N слов по измерению"""
        if len(vectors) == 0:
            return []

        indices = np.argsort(vectors[:, dim])
        if not ascending:
            indices = indices[::-1]
        return [words[i] for i in indices[:n_words]]

    def print_covariance_analysis(self, results):
        """Улучшенный вывод результатов анализа ковариаций"""
        print("\nРезультаты анализа различий между кластерами:")
        print("(KL-дивергенция измеряет различия в распределениях значений по измерениям)")

        for res in results:
            print(f"\nИзмерение {res['dimension']}")
            print(f"KL-дивергенция: {res['kl_divergence']:.3f}")
            print(f"Средние значения: кластер {res['cluster1']} = {res['mean1']:.2f}, "
                  f"кластер {res['cluster2']} = {res['mean2']:.2f}, разница = {res['mean_diff']:.2f}")

            print(f"\nКластер {res['cluster1']} (компонента {res['component1']}):")
            print(f"  Топ: {', '.join(res['cluster1_top'][:5])}")
            print(f"  Антонимы: {', '.join(res['cluster1_opposite'][:3])}")

            print(f"\nКластер {res['cluster2']} (компонента {res['component2']}):")
            print(f"  Топ: {', '.join(res['cluster2_top'][:5])}")
            print(f"  Антонимы: {', '.join(res['cluster2_opposite'][:3])}")

            print("\n" + "-"*50)

    def print_variance_analysis(self, results):
        """Улучшенный вывод результатов анализа дисперсий"""
        print("\nРезультаты анализа дисперсий:")
        print("(Относительная дисперсия показывает значимость измерения в кластере по сравнению с общим распределением)")

        for res in results:
            print(f"\nИзмерение {res['dimension']}")
            print(f"Дисперсия: {res['variance']:.3f}")
            print(f"Относительная дисперсия: {res['relative_variance']:.2f}x")

            print(f"\nТоп слова: {', '.join(res['top_words'][:5])}")
            print(f"Антонимы: {', '.join(res['opposite_words'][:3])}")

            print("\nНаиболее коррелированные измерения:")
            for dim, corr in zip(res['correlated_dims'], res['correlation_values']):
                print(f"  Измерение {dim}: коэффициент корреляции = {corr:.2f}")

            print("\n" + "-"*50)

    def compare_clusters_visually(self, cluster_indices, max_points=500, label_top_n=20):
        """Визуализация сравнения кластеров с подписями точек"""
        if not all(idx in self.sub_level_gmms for idx in cluster_indices):
            raise ValueError("Некоторые кластеры не существуют или не были подкластеризованы")

        # Собираем маску для выбранных кластеров
        combined_mask = np.zeros(len(self.reduced_vectors), dtype=bool)
        for idx in cluster_indices:
            combined_mask |= (self.gmm.predict(self.reduced_vectors) == idx)

        # Создаем mapper: глобальный индекс -> локальный индекс
        index_mapper = np.cumsum(combined_mask) - 1
        index_mapper[~combined_mask] = -1

        # Подготовка данных
        points = self.reduced_vectors[combined_mask]
        words = [self.words[i] for i in np.where(combined_mask)[0]]
        total_points = len(points)

        # Инициализация меток
        combined_labels = np.zeros(total_points, dtype=int)
        offset = 0

        for idx in cluster_indices:
            # Получаем точки кластера
            cluster_mask = (self.gmm.predict(self.reduced_vectors) == idx)
            global_indices = np.where(cluster_mask)[0]
            local_indices = index_mapper[global_indices]

            # Фильтруем невалидные индексы
            valid_mask = (local_indices != -1)
            local_indices = local_indices[valid_mask]

            # Получаем подкластеры
            gmm_sub = self.sub_level_gmms[idx]
            sub_labels = gmm_sub.predict(self.reduced_vectors[cluster_mask][valid_mask])

            # Обновляем метки
            combined_labels[local_indices] = sub_labels + offset
            offset += gmm_sub.n_components + 1

        # t-SNE проекция
        tsne = TSNE(n_components=2, random_state=42)
        embedded = tsne.fit_transform(points)

        # Создаем фигуру
        plt.figure(figsize=(20, 16))

        # Цвета для кластеров
        colors = plt.cm.tab20(np.linspace(0, 1, offset))

        # Рисуем точки
        scatter = plt.scatter(
            embedded[:, 0], embedded[:, 1],
            c=[colors[l] for l in combined_labels],
            alpha=0.7, s=50
        )

        # Добавляем подписи для наиболее характерных точек каждого подкластера
        texts = []
        for label in np.unique(combined_labels):
            mask = combined_labels == label
            cluster_points = embedded[mask]
            cluster_words = [words[i] for i in np.where(mask)[0]]

            # Находим центр подкластера
            center = np.median(cluster_points, axis=0)

            # Выбираем ближайшие точки к центру
            distances = np.linalg.norm(cluster_points - center, axis=1)
            closest_indices = np.argsort(distances)[:label_top_n]

            for idx in closest_indices:
                texts.append(plt.text(
                    cluster_points[idx, 0], cluster_points[idx, 1],
                    cluster_words[idx],
                    fontsize=8,
                    bbox=dict(
                        facecolor='white',
                        alpha=0.7,
                        edgecolor=colors[label],
                        boxstyle='round,pad=0.2'
                    )
                ))

        # Автоматическая регулировка подписей
        adjust_text(
            texts,
            arrowprops=dict(
                arrowstyle='-',
                color='gray',
                lw=0.5,
                alpha=0.5
            ),
            expand_points=(1.2, 1.2),
            expand_text=(1.1, 1.1),
            force_text=(0.5, 0.5),
            force_points=(0.8, 0.8),
            lim=1000
        )

        # Настройка легенды
        legend_elements = [
            plt.Line2D([0], [0], marker='o', color='w',
                      markerfacecolor=colors[i], markersize=10,
                      label=f'Кластер {i}')
            for i in np.unique(combined_labels)
        ]

        plt.legend(
            handles=legend_elements,
            title='Подкластеры',
            bbox_to_anchor=(1.05, 1),
            loc='upper left'
        )

        plt.title(
            f"Сравнение кластеров: {', '.join(map(str, cluster_indices))}\n"
            f"Подписаны топ-{label_top_n} слов каждого подкластера",
            pad=20
        )
        plt.grid(True, alpha=0.2)
        plt.tight_layout()
        plt.show()

In [ ]:
if __name__ == "__main__":
    # Инициализация анализатора
    analyzer = Word2VecSemanticAnalyzer(selected_vectors, selected_nouns)

    # 1. Снижение размерности с автоматическим выбором компонент
    analyzer.reduce_dimensions(n_components=200, variance_threshold=0.0025)

    # 2. Верхнеуровневая кластеризация
    analyzer.find_optimal_clusters(max_k=100, min_k=50, n_init=10, step=5)

    # 3. Визуализация и анализ кластеров верхнего уровня
    analyzer.visualize_clusters(max_points=2000)
    analyzer.print_cluster_words(top_n=15)
    quality_report = analyzer.analyze_cluster_quality()

    # 4. Иерархическая подкластеризация (только для крупных кластеров)
    analyzer.hierarchical_clustering(max_subclusters=10, min_cluster_size=60)

    # 5. Углубленный анализ кластеров
    if len(analyzer.sub_level_gmms) >= 2:
        valid_clusters = [k for k in analyzer.sub_level_gmms.keys()]

        if quality_report and 'coherence' in quality_report:
            cluster_coherence = quality_report['coherence']
            # Фильтруем только кластеры с подкластеризацией
            filtered_clusters = [(k, v) for k, v in cluster_coherence.items() if k in valid_clusters]

            if len(filtered_clusters) >= 2:
                sorted_clusters = sorted(filtered_clusters, key=lambda x: x[1])
                cluster1 = sorted_clusters[0][0]
                cluster2 = sorted_clusters[-1][0]
            else:
                cluster1, cluster2 = valid_clusters[:2]
        else:
            cluster1, cluster2 = valid_clusters[:2]

        print(f"\nСравниваем кластеры {cluster1} и {cluster2}")
        if quality_report and 'coherence' in quality_report:
            print(f"Кластер {cluster1} (когерентность: {quality_report['coherence'].get(cluster1, 'N/A'):.3f})")
            print(f"Кластер {cluster2} (когерентность: {quality_report['coherence'].get(cluster2, 'N/A'):.3f}")
        print("="*70)

        # Анализ различий в ковариациях
        try:
            cov_results = analyzer.analyze_covariance_diff(cluster1, cluster2)
            analyzer.print_covariance_analysis(cov_results)
        except Exception as e:
            print(f"Ошибка при анализе ковариаций: {str(e)}")

        # Анализ дисперсий для кластеров
        try:
            print("\n" + "="*70)
            print(f"Анализ структуры кластера {cluster1}:")
            var_results1 = analyzer.analyze_variance(cluster1)
            analyzer.print_variance_analysis(var_results1)

            print("\n" + "="*70)
            print(f"Анализ структуры кластера {cluster2}:")
            var_results2 = analyzer.analyze_variance(cluster2)
            analyzer.print_variance_analysis(var_results2)
        except Exception as e:
            print(f"Ошибка при анализе дисперсий: {str(e)}")

        # Сравнительная визуализация
        try:
            analyzer.compare_clusters_visually([cluster1, cluster2])
        except Exception as e:
            print(f"Ошибка при визуализации: {str(e)}")
    else:
        print("\nНедостаточно кластеров для сравнения (нужно минимум 2 подкластеризованных кластера)")

In [ ]:
!pip install pymorphy3 nltk transformers datasets plotly scikit-learn adjustText

In [ ]:
!wget http://opencorpora.org/files/export/annot/annot.opcorpora.xml.zip
!unzip annot.opcorpora.xml.zip

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
import torch
import matplotlib.pyplot as plt
import plotly.express as px
from transformers import AutoTokenizer, AutoModel
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, davies_bouldin_score
from tqdm import tqdm
from collections import defaultdict, Counter
from adjustText import adjust_text
from scipy.stats import entropy
from pymorphy3 import MorphAnalyzer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from multiprocessing import Pool
import gc
from collections import defaultdict

def find_noun_positions(sent, noun_set):
    sent_lower = sent.lower()
    positions = defaultdict(list)
    for noun in noun_set:
        for match in re.finditer(r'\b' + re.escape(noun) + r'\b', sent_lower):
            positions[noun].append((match.start(), match.end()))
    return positions

class BertSemanticAnalyzer:
    def __init__(self, model_name="ai-forever/ruBert-base"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.max_seq_length = 128
        self.vectors = None
        self.vocab = None
        self.word_to_idx = None
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        self.model.eval()
        self.random_state = 42
        self.morph = MorphAnalyzer()
        self.pos_cache = {}
        self.stopwords = set(stopwords.words('russian'))
        self.punctuation = set(string.punctuation + '«»—')
        self.sub_level_gmms = {}
        self.stationary_indices = defaultdict(list)

    def extract_sentences(self, file_path):
        try:
            tree = ET.parse(file_path)
            return [text.text.strip() for text in tree.findall(".//source") if text.text]
        except Exception as e:
            print(f"Ошибка чтения файла: {e}")
            return []

    def extract_frequent_nouns(self, sentences, top_n=10000):
        counter = Counter()
        for sent in tqdm(sentences, desc="Извлечение существительных"):
            words = re.findall(r'\b[\w-]+\b', sent.lower())
            for word in words:
                if word in self.stopwords or len(word) < 2:
                    continue

                if word not in self.pos_cache:
                    parsed = self.morph.parse(word)
                    if parsed:
                        best = max(parsed, key=lambda p: p.score)
                        self.pos_cache[word] = {
                            'lemma': best.normal_form,
                            'pos': best.tag.POS
                        }

                info = self.pos_cache.get(word)
                if info and info['pos'] == 'NOUN':
                    counter[info['lemma']] += 1

        return [lemma for lemma, _ in counter.most_common(top_n)]

    def get_contextual_embeddings(self, sentences, nouns, batch_size=64, max_contexts_per_noun=50, sample_size=None):
        noun_set = set(nouns)

        # Если указана подвыборка корпуса, берем случайную часть
        if sample_size is not None and sample_size < len(sentences):
            import random
            random.seed(self.random_state)
            sentences = random.sample(sentences, sample_size)

        noun_pattern = re.compile(r'\b(' + '|'.join(map(re.escape, nouns)) + r')\b')
        relevant_sents = [s for s in sentences if noun_pattern.search(s.lower())]

        # Предварительная индексация существительных с ограничением вхождений
        noun_indices = defaultdict(list)
        counts = defaultdict(int)

        print("Индексация существительных...")
        with Pool() as pool:
            results = list(
                tqdm(
                    pool.starmap(find_noun_positions, [(sent, noun_set) for sent in relevant_sents]),
                    total=len(relevant_sents),
                    desc="Индексация предложений"
                )
            )

        for i, positions in enumerate(results):
            for noun, spans in positions.items():
                if counts[noun] < max_contexts_per_noun:
                    noun_indices[noun].append((i, spans))
                    counts[noun] += 1

        # Обработка батчей
        embeddings = defaultdict(list)
        total_batches = (len(relevant_sents) + batch_size - 1) // batch_size

        print("Извлечение контекстуальных эмбеддингов...")
        for batch_idx in tqdm(range(total_batches), desc="Обработка контекстов"):
            start = batch_idx * batch_size
            end = (batch_idx + 1) * batch_size
            batch = relevant_sents[start:end]

            inputs = self.tokenizer(
                batch,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=self.max_seq_length,
                return_offsets_mapping=True
            ).to(self.device)

            with torch.no_grad():
                model_inputs = {k: v for k, v in inputs.items() if k != "offset_mapping"}
                outputs = self.model(**model_inputs)
                hidden_states = outputs.last_hidden_state.cpu().numpy()

            for sent_idx in range(len(batch)):
                offset_mapping = inputs.offset_mapping[sent_idx].cpu().numpy()

                for noun, positions in noun_indices.items():
                    for global_idx, spans in positions:
                        if global_idx != start + sent_idx:
                            continue

                        token_indices = []
                        for start_char, end_char in spans:
                            for tok_idx, (tok_start, tok_end) in enumerate(offset_mapping):
                                if tok_start >= start_char and tok_end <= end_char:
                                    token_indices.append(tok_idx)

                        if token_indices:
                            emb = hidden_states[sent_idx, token_indices].mean(axis=0)
                            embeddings[noun].append(emb)

            del inputs, outputs, hidden_states
            torch.cuda.empty_cache()
            gc.collect()

        self._create_final_embeddings(embeddings, nouns)
        return self.vectors

    def _create_final_embeddings(self, embeddings, nouns):
        self.vocab = []
        self.vectors = []
        for noun in nouns:
            if noun in embeddings and len(embeddings[noun]) > 0:
                self.vocab.append(noun)
                self.vectors.append(np.mean(embeddings[noun], axis=0))

        self.vectors = np.array(self.vectors)
        self.word_to_idx = {w: i for i, w in enumerate(self.vocab)}
        print(f"Получено {len(self.vectors)} контекстуальных эмбеддингов")

    def reduce_dimensions(self, n_components=200, variance_threshold=0.01):
        pca = PCA(n_components=min(n_components, self.vectors.shape[1]),
                random_state=self.random_state)
        self.reduced_vectors = pca.fit_transform(self.vectors)

        cum_var = np.cumsum(pca.explained_variance_ratio_)
        optimal_components = np.where(pca.explained_variance_ratio_ > variance_threshold)[0].size
        optimal_components = max(150, optimal_components)

        plt.figure(figsize=(12,6))
        plt.subplot(121)
        plt.plot(pca.explained_variance_ratio_, 'o-')
        plt.axvline(optimal_components, color='r', linestyle='--')
        plt.title('Объясненная дисперсия по компонентам')
        plt.xlabel('Номер компоненты')
        plt.ylabel('Доля объясненной дисперсии')

        plt.subplot(122)
        plt.plot(cum_var, 'o-')
        plt.axvline(optimal_components, color='r', linestyle='--')
        plt.axhline(cum_var[optimal_components], color='g', linestyle='--')
        plt.title('Накопленная объясненная дисперсия')
        plt.xlabel('Число компонент')
        plt.ylabel('Накопленная дисперсия')

        plt.tight_layout()
        plt.show()

        if optimal_components != n_components:
            print(f"Оптимальное число компонент: {optimal_components}")
            pca = PCA(n_components=optimal_components, random_state=self.random_state)
            self.reduced_vectors = pca.fit_transform(self.vectors)
        return self.reduced_vectors


    def find_optimal_clusters(self, max_k=150, min_k=50, step=5, n_init=5):
        cluster_range = range(min_k, max_k + 1, step)
        metrics = {
            'silhouette': [],
            'davies_bouldin': [],
            'combo': [],
            'bic': [],
            'aic': []
        }

        best_metrics = {
            'silhouette': -np.inf,
            'davies_bouldin': np.inf,
            'combo': -np.inf,
            'k': None
        }

        for k in tqdm(cluster_range, desc="Кластеризация"):
            try:
                gmm = GaussianMixture(
                    n_components=k,
                    covariance_type='diag',
                    random_state=self.random_state,
                    n_init=n_init,
                    max_iter=500,
                    tol=1e-4,
                    reg_covar=1e-3
                )
                gmm.fit(self.reduced_vectors)

                if not gmm.converged_:
                    raise ValueError("GMM не сошелся")

                labels = gmm.predict(self.reduced_vectors)

                sil_score = silhouette_score(self.reduced_vectors, labels)
                db_score = davies_bouldin_score(self.reduced_vectors, labels)
                combo_score = 0.7*sil_score - 0.3*db_score

                if combo_score > best_metrics['combo']:
                    best_metrics.update({
                        'silhouette': sil_score,
                        'davies_bouldin': db_score,
                        'combo': combo_score,
                        'k': k
                    })
                    self.optimal_k = k
                    self.gmm = gmm

                metrics['silhouette'].append(sil_score)
                metrics['davies_bouldin'].append(db_score)
                metrics['combo'].append(combo_score)
                metrics['bic'].append(gmm.bic(self.reduced_vectors))
                metrics['aic'].append(gmm.aic(self.reduced_vectors))

            except Exception as e:
                print(f"Ошибка для k={k}: {str(e)}")
                for name in metrics.keys():
                    metrics[name].append(None)

        self._plot_cluster_metrics(cluster_range, metrics)
        print(f"\nОптимальное число кластеров: {self.optimal_k}")
        print(f"Silhouette: {best_metrics['silhouette']:.3f}")
        print(f"Davies-Bouldin: {best_metrics['davies_bouldin']:.3f}")
        return self.optimal_k

    def _plot_cluster_metrics(self, cluster_range, metrics):
        plt.figure(figsize=(18,10))

        valid_metrics = {name: [(x is not None) for x in vals] for name, vals in metrics.items()}

        plt.subplot(2,2,1)
        if any(valid_metrics['silhouette']):
            plt.plot(cluster_range, metrics['silhouette'], 'o-')
            plt.axvline(self.optimal_k, color='r', linestyle='--')
        plt.title('Silhouette Score')
        plt.xlabel('Число кластеров')
        plt.ylabel('Значение метрики')

        plt.subplot(2,2,2)
        if any(valid_metrics['davies_bouldin']):
            plt.plot(cluster_range, metrics['davies_bouldin'], 'o-', color='orange')
            plt.axvline(self.optimal_k, color='r', linestyle='--')
        plt.title('Davies-Bouldin Index')
        plt.xlabel('Число кластеров')
        plt.ylabel('Значение метрики')

        plt.subplot(2,2,3)
        if any(valid_metrics['combo']):
            plt.plot(cluster_range, metrics['combo'], 'o-', color='green')
            plt.axvline(self.optimal_k, color='r', linestyle='--')
        plt.title('Комбинированная метрика')
        plt.xlabel('Число кластеров')
        plt.ylabel('Значение метрики')

        plt.subplot(2,2,4)
        if any(valid_metrics['bic']) and any(valid_metrics['aic']):
            plt.plot(cluster_range, metrics['bic'], 'o-', label='BIC')
            plt.plot(cluster_range, metrics['aic'], 'o-', label='AIC')
            plt.axvline(self.optimal_k, color='r', linestyle='--')
            plt.legend()
        plt.xlabel('Число кластеров')
        plt.ylabel('Значение метрики')

        plt.tight_layout()
        plt.show()

    def visualize_clusters(self, max_points=1000, perplexity=30):
        if self.gmm is None:
            raise ValueError("Сначала выполните кластеризацию!")

        labels = self.gmm.predict(self.reduced_vectors)
        cluster_sizes = np.bincount(labels)
        probs = self.gmm.predict_proba(self.reduced_vectors)
        uncertainties = 1 - np.max(probs, axis=1)

        if len(self.reduced_vectors) > max_points:
            confident_indices = np.argsort(-uncertainties)[:max_points]
            sample_vectors = self.reduced_vectors[confident_indices]
            sample_words = [self.vocab[i] for i in confident_indices]
            sample_labels = labels[confident_indices]
            sample_uncertainties = uncertainties[confident_indices]
        else:
            sample_vectors = self.reduced_vectors
            sample_words = self.vocab
            sample_labels = labels
            sample_uncertainties = uncertainties

        perplexity = min(perplexity, len(sample_vectors) // 3 - 1)
        tsne = TSNE(
            n_components=2,
            random_state=self.random_state,
            perplexity=perplexity,
            init='pca',
            learning_rate='auto'
        )
        tsne_vectors = tsne.fit_transform(sample_vectors)

        fig = px.scatter(
            x=tsne_vectors[:, 0], y=tsne_vectors[:, 1],
            color=sample_labels.astype(str),
            size=1-sample_uncertainties,
            hover_name=sample_words,
            title=(
                f"Семантические кластеры (k={self.optimal_k})<br>"
                f"Всего слов: {len(self.vocab)} | Средний размер кластера: {np.mean(cluster_sizes):.1f}<br>"
                f"Размер точки отражает уверенность кластеризации"
            ),
            width=1000, height=800,
            color_discrete_sequence=px.colors.qualitative.Alphabet
        )
        fig.update_traces(marker=dict(line=dict(width=0.5, color='DarkSlateGrey')))
        fig.show()

        self._plot_static_clusters(tsne_vectors, sample_labels, sample_words, sample_uncertainties)

    def _plot_static_clusters(self, tsne_vectors, labels, words, uncertainties):
        plt.figure(figsize=(20,15))
        scatter = plt.scatter(
            tsne_vectors[:,0], tsne_vectors[:,1],
            c=labels, cmap='tab20',
            s=100*(1-uncertainties),
            alpha=0.7,
            edgecolor='k',
            linewidths=0.5
        )

        texts = []
        for cluster in np.unique(labels):
            mask = labels == cluster
            center = np.median(tsne_vectors[mask], axis=0)
            distances = np.linalg.norm(tsne_vectors[mask] - center, axis=1)
            closest_idx = np.argsort(distances)[:5]

            for idx in closest_idx:
                texts.append(plt.text(
                    tsne_vectors[mask][idx,0], tsne_vectors[mask][idx,1],
                    words[np.where(mask)[0][idx]],
                    fontsize=9,
                    bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1)
                ))

        adjust_text(texts)
        plt.legend(
            handles=scatter.legend_elements()[0],
            labels=[f"Кластер {i}" for i in np.unique(labels)],
            title="Кластеры",
            bbox_to_anchor=(1,1),
            loc='upper left'
        )
        plt.title(
            f"Семантические кластеры (k={self.optimal_k})\n"
            f"Размер точки отражает уверенность кластеризации\n"
            f"Всего точек: {len(tsne_vectors)}",
            pad=20
        )
        plt.axis('off')
        plt.tight_layout()
        plt.show()

    def hierarchical_clustering(self, max_subclusters=10, min_subclusters=3,
                           max_points=200, min_cluster_size=20):
        cluster_sizes = np.bincount(self.gmm.predict(self.reduced_vectors))

        for cluster_idx in range(self.optimal_k):
            size = cluster_sizes[cluster_idx]
            if size < min_cluster_size:
                print(f"Кластер {cluster_idx} слишком мал ({size} точек), пропускаем")
                continue

            cluster_mask = self.gmm.predict(self.reduced_vectors) == cluster_idx
            cluster_points = self.reduced_vectors[cluster_mask]

            if np.isnan(cluster_points).any():
                print(f"Кластер {cluster_idx} содержит NaN значения, пропускаем")
                continue

            self._process_single_cluster(
                cluster_idx,
                max_subclusters,
                min_subclusters,
                max_points
            )

    def _process_single_cluster(self, cluster_idx, max_subclusters, min_subclusters, max_points):
        """Обработка одного кластера верхнего уровня с улучшенным анализом"""
        cluster_mask = self.gmm.predict(self.reduced_vectors) == cluster_idx
        cluster_points = self.reduced_vectors[cluster_mask]
        cluster_words = [self.vocab[i] for i in np.where(cluster_mask)[0]]

        print(f"\n{'='*50}")
        print(f"Анализ кластера {cluster_idx} ({len(cluster_points)} слов)")
        print(f"Примеры слов: {', '.join(np.random.choice(cluster_words, min(10, len(cluster_words))))}")
        print("="*50)


        # Определение оптимального числа подкластеров
        optimal_sub_k, metrics = self._find_optimal_subclusters(
            cluster_points, max_subclusters, min_subclusters
        )

        if optimal_sub_k < 2:
            print("Не удалось найти значимые подкластеры")
            return

        # Кластеризация и визуализация
        gmm_sub, sub_labels, original_indices = self._perform_subclustering(
        cluster_points, cluster_words, cluster_idx, optimal_sub_k, max_points
        )

        self._store_subcluster_results(cluster_idx, cluster_mask, sub_labels, gmm_sub, original_indices)

        # Визуализация метрик
        self._plot_subcluster_metrics(metrics, optimal_sub_k)

        # Анализ семантической когерентности подкластеров
        self._analyze_subclusters_semantics(cluster_words, sub_labels)

    def _find_optimal_subclusters(self, points, max_k, min_k):
        """Улучшенный метод определения оптимального числа подкластеров"""
        sub_range = range(min_k, max_k+1)
        metrics = {
            'k': [],
            'silhouette': [],
            'davies_bouldin': [],
            'bic': [],
            'combo': [],
            'gap': []
        }

        best_k = min_k
        best_combo = -np.inf
        prev_sil = -1

        for k in sub_range:
            metrics['k'].append(k)
            if len(points) < k*2:
                print(f"Слишком мало точек ({len(points)}) для k={k}")
                for m in ['silhouette', 'davies_bouldin', 'bic', 'combo', 'gap']:
                    metrics[m].append(None)
                continue

            try:
                gmm = GaussianMixture(
                    n_components=k,
                    covariance_type='diag',
                    random_state=self.random_state,
                    n_init=5,
                    reg_covar=1e-1,
                    tol=1e-4,
                    max_iter=500
                )
                gmm.fit(points)
                labels = gmm.predict(points)

                unique_labels = np.unique(labels)
                if len(unique_labels) < k:
                    raise ValueError(f"Обнаружены пустые кластеры для k={k}")

                sil_score = silhouette_score(points, labels) if k > 1 else 0
                db_score = davies_bouldin_score(points, labels) if k > 1 else np.inf
                bic = gmm.bic(points)
                combo_score = 0.7*sil_score - 0.3*db_score

                try:
                    ref_disps = []
                    for _ in range(3):
                        # Генерация синтетических данных
                        random_data = np.random.rand(*points.shape) * (points.max(0)-points.min(0)) + points.min(0)

                        # Фильтрация NaN/Inf
                        random_data = np.nan_to_num(random_data, nan=0.0, posinf=1e6, neginf=-1e6)

                        # Вычисление логарифма с защитой
                        gmm_ref = GaussianMixture(
                            n_components=k,
                            random_state=self.random_state,
                            reg_covar=1e-1
                        )
                        gmm_ref.fit(random_data)
                        log_prob = gmm_ref.score(random_data)
                        ref_disps.append(log_prob if log_prob > -np.inf else 0)

                    current_score = gmm.score(points)
                    gap = np.mean(ref_disps) - (current_score if current_score > -np.inf else 0)
                except Exception as e:
                    print(f"Ошибка при вычислении Gap Statistic: {str(e)}")
                    gap = 0

                metrics['silhouette'].append(sil_score)
                metrics['davies_bouldin'].append(db_score)
                metrics['bic'].append(bic)
                metrics['combo'].append(combo_score)
                metrics['gap'].append(gap)

                # Обновление лучшего k
                if combo_score > best_combo and gap > 0:
                    best_combo = combo_score
                    best_k = k

                prev_sil = sil_score

            except Exception as e:
                print(f"Ошибка для k={k}: {str(e)}")
                for m in ['silhouette', 'davies_bouldin', 'bic', 'combo', 'gap']:
                    metrics[m].append(None)

        return best_k, metrics

    def _perform_subclustering(self, points, words, cluster_idx, optimal_k, max_points=500):
        """Улучшенная визуализация подкластеров с подписями всех точек"""
        gmm_sub = GaussianMixture(
                n_components=optimal_k,
                covariance_type='diag',
                random_state=self.random_state,
                n_init=5,
                reg_covar=1e-1,
                tol=1e-4,
                max_iter=500
            )
        sub_labels = gmm_sub.fit_predict(points)

        # Проверка на NaN в метках
        if np.isnan(sub_labels).any():
            raise ValueError("Обнаружены NaN в метках кластеров")

        # Добавляем расчет неопределенности
        probs = gmm_sub.predict_proba(points)
        uncertainties = 1 - np.max(probs, axis=1)

        # Сохраняем оригинальные индексы до выборки
        original_indices = np.arange(len(points))

        # Ограничиваем количество точек для визуализации
        if len(points) > max_points:
            indices = np.random.choice(len(points), max_points, replace=False)
            points = points[indices]
            words = [words[i] for i in indices]
            sub_labels = sub_labels[indices]
            uncertainties = uncertainties[indices]
            original_indices = original_indices[indices]

        # t-SNE проекция
        perplexity = min(50, len(points)-1)
        tsne = TSNE(
            n_components=2,
            perplexity=perplexity,
            random_state=self.random_state,
            init='pca',
            learning_rate='auto'
        )
        embedded = tsne.fit_transform(points)

        # Создаем фигуру
        plt.figure(figsize=(20, 15))

        # Размер точек зависит от уверенности кластеризации (1 - uncertainty)
        sizes = 50 + 150 * (1 - uncertainties)

        # Цвета для подкластеров
        colors = plt.cm.tab20(np.linspace(0, 1, optimal_k))

        # Рисуем точки с подписями
        texts = []
        for i in range(len(points)):
            plt.scatter(
                embedded[i, 0], embedded[i, 1],
                color=colors[sub_labels[i]],
                s=sizes[i],
                alpha=0.7,
                edgecolor='k',
                linewidths=0.3
            )
            texts.append(plt.text(
                embedded[i, 0], embedded[i, 1],
                words[i],
                fontsize=8,
                color='black',
                ha='center', va='center'
            ))

        # Настройка легенды
        legend_elements = [
            plt.Line2D([0], [0], marker='o', color='w',
                      markerfacecolor=colors[i], markersize=10,
                      label=f'Подкластер {i}')
            for i in range(optimal_k)
        ]

        plt.legend(
            handles=legend_elements,
            title='Подкластеры',
            bbox_to_anchor=(1.05, 1),
            loc='upper left'
        )

        # Настройка заголовка
        plt.title(
            f"Разбиение кластера {cluster_idx} на {optimal_k} подкластеров\n"
            f"Размер точки отражает уверенность кластеризации\n"
            f"Всего точек: {len(points)}",
            pad=20
        )

        # Автоматическая регулировка подписей
        adjust_text(
            texts,
            arrowprops=dict(arrowstyle='-', color='gray', lw=0.5),
            expand_points=(1.2, 1.2),
            expand_text=(1.1, 1.1),
            force_text=(0.5, 0.5),
            force_points=(0.8, 0.8),
            lim=1000
        )

        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

        return gmm_sub, sub_labels, original_indices

    def _plot_subcluster_metrics(self, metrics, optimal_k):
        """Исправленная визуализация метрик подкластеров"""
        plt.figure(figsize=(15,10))

        # Фильтруем невалидные значения
        valid = ~np.isnan(metrics['silhouette'])
        k_values = np.array(metrics['k'])[valid]

        plt.subplot(2,2,1)
        plt.plot(k_values, np.array(metrics['silhouette'])[valid], 'o-')
        plt.axvline(optimal_k, color='r', linestyle='--')
        plt.title('Silhouette Score')

        plt.subplot(2,2,2)
        plt.plot(k_values, np.array(metrics['davies_bouldin'])[valid], 'o-', color='orange')
        plt.axvline(optimal_k, color='r', linestyle='--')
        plt.title('Davies-Bouldin Index')

        plt.subplot(2,2,3)
        plt.plot(k_values, np.array(metrics['combo'])[valid], 'o-', color='green')
        plt.axvline(optimal_k, color='r', linestyle='--')
        plt.title('Комбинированная метрика')

        plt.subplot(2,2,4)
        plt.plot(k_values, np.array(metrics['gap'])[valid], 'o-', color='purple')
        plt.axvline(optimal_k, color='r', linestyle='--')
        plt.title('Gap Statistic')

        plt.tight_layout()
        plt.show()

    def _store_subcluster_results(self, cluster_idx, mask, sub_labels, gmm_sub, original_indices):
        self.sub_level_gmms[cluster_idx] = gmm_sub
        cluster_global_indices = np.where(mask)[0]

        for sub_id in range(gmm_sub.n_components):
            sub_mask = sub_labels == sub_id
            global_indices = cluster_global_indices[original_indices[sub_mask]]
            for idx in global_indices:
                self.stationary_indices[idx].append((cluster_idx, sub_id))

    def _analyze_subclusters_semantics(self, words, labels):
        unique_labels = np.unique(labels)
        print("\nСемантический анализ подкластеров:")

        for sub_id in unique_labels:
            cluster_words = [w for w, l in zip(words, labels) if l == sub_id]
            print(f"\nПодкластер {sub_id} ({len(cluster_words)} слов):")
            print(", ".join(cluster_words[:15]) + ("..." if len(cluster_words) > 15 else ""))

    def get_cluster_words(self, top_n=10, sort_clusters=True):
        """Получение топ-слов с вероятностями и сортировкой по кластерам"""
        cluster_words = defaultdict(list)
        probs = self.gmm.predict_proba(self.reduced_vectors)

        for word, label, prob in zip(self.vocab,
                                   self.gmm.predict(self.reduced_vectors),
                                   probs):
            cluster_words[label].append((word, np.max(prob)))

        result = {}
        for k in cluster_words:
            sorted_words = sorted(cluster_words[k], key=lambda x: -x[1])
            result[k] = [w[0] for w in sorted_words[:top_n]]

        return dict(sorted(result.items())) if sort_clusters else result

    def print_cluster_words(self, top_n=15):
        """Улучшенный вывод с информацией о размерах и вероятностях"""
        cluster_words = self.get_cluster_words(top_n=top_n)
        labels = self.gmm.predict(self.reduced_vectors)
        cluster_sizes = np.bincount(labels)
        probs = self.gmm.predict_proba(self.reduced_vectors)

        print("\nТоп слов по кластерам (размер | средняя вероятность):")
        for cluster in sorted(cluster_words.keys()):
            cluster_probs = probs[labels == cluster]
            avg_prob = np.mean(np.max(cluster_probs, axis=1)) if len(cluster_probs) > 0 else 0

            print(f"\nКластер {cluster} [размер: {cluster_sizes[cluster]} | prob: {avg_prob:.2f}]:")
            print(", ".join(cluster_words[cluster][:top_n]))

    def analyze_cluster_quality(self, sample_size=1000):
        labels = self.gmm.predict(self.reduced_vectors)
        cluster_coherence = {}
        cluster_diversity = {}
        cluster_sizes = np.bincount(labels)

        for cluster in np.unique(labels):
            mask = labels == cluster
            cluster_vectors = self.vectors[mask]
            cluster_words = [self.vocab[i] for i in np.where(mask)[0]]

            similarities = cosine_similarity(cluster_vectors)
            np.fill_diagonal(similarities, 0)
            cluster_coherence[cluster] = np.mean(similarities)

            diversities = []
            for word in cluster_words[:sample_size]:
                word_vec = self.vectors[self.word_to_idx[word]]
                sims = cosine_similarity([word_vec], self.vectors)[0]
                top_indices = np.argsort(-sims)[1:11]
                top_words = [self.vocab[i] for i in top_indices]
                counts = [cluster_words.count(w) for w in top_words]
                probs = np.array(counts) / len(cluster_words)
                probs = probs[probs > 0]
                diversities.append(entropy(probs))

            cluster_diversity[cluster] = np.mean(diversities) if diversities else 0

        self._plot_cluster_quality(cluster_coherence, cluster_diversity, cluster_sizes)
        return {
            'coherence': cluster_coherence,
            'diversity': cluster_diversity,
            'cluster_sizes': cluster_sizes
        }

    def _plot_cluster_quality(self, coherence, diversity, sizes):
        """Обновленная визуализация с размерами кластеров"""
        plt.figure(figsize=(18,6))
        clusters = sorted(coherence.keys())

        # График когерентности с цветовой индикацией размеров
        plt.subplot(131)
        max_size = max(sizes)
        colors = [plt.cm.viridis(sizes[c]/max_size) for c in clusters]  # Исправлено здесь

        bars = plt.bar(clusters,
                      [coherence[c] for c in clusters],
                      color=colors,
                      alpha=0.7)

        # Аннотации с размерами
        for idx, bar in enumerate(bars):
            plt.text(bar.get_x() + bar.get_width()/2,
                    bar.get_height(),
                    f'{sizes[idx]}',
                    ha='center',
                    va='bottom',
                    fontsize=8)

        plt.title('Когерентность и размер кластеров\n(цвет отражает относительный размер)')
        plt.xlabel('Кластер')
        plt.ylabel('Когерентность')

        plt.subplot(132)
        plt.bar(clusters, [diversity[c] for c in clusters], color='orange', alpha=0.7)
        plt.title('Диверсификация кластеров')

        plt.subplot(133)
        plt.bar(clusters, [sizes[c] for c in clusters], color='green', alpha=0.7)
        plt.yscale('log')
        plt.title('Размеры кластеров (лог. шкала)')

        plt.tight_layout()
        plt.show()


    def analyze_covariance_diff(self, cluster_idx1, cluster_idx2, top_n=5, top_k=10):
        gmm1 = self.sub_level_gmms[cluster_idx1]
        gmm2 = self.sub_level_gmms[cluster_idx2]

        mask1 = self.gmm.predict(self.reduced_vectors) == cluster_idx1
        mask2 = self.gmm.predict(self.reduced_vectors) == cluster_idx2
        points1 = self.reduced_vectors[mask1]
        points2 = self.reduced_vectors[mask2]
        words1 = [self.vocab[i] for i in np.where(mask1)[0]]
        words2 = [self.vocab[i] for i in np.where(mask2)[0]]

        kl_divs = []
        for dim in range(self.reduced_vectors.shape[1]):
            hist1, _ = np.histogram(points1[:, dim], bins=20, density=True)
            hist2, _ = np.histogram(points2[:, dim], bins=20, density=True)

            hist1 = (hist1 + 1e-10) / (hist1.sum() + 1e-10)
            hist2 = (hist2 + 1e-10) / (hist2.sum() + 1e-10)

            kl_div = entropy(hist1, hist2) + entropy(hist2, hist1)
            kl_divs.append(kl_div)

        top_dims = np.argsort(-np.array(kl_divs))[:top_n]

        results = []
        for dim in top_dims:
            comp1_words = [self.vocab[i] for i in np.where(mask1)[0]]
            comp2_words = [self.vocab[i] for i in np.where(mask2)[0]]

            top1 = self._get_top_words_by_dim(comp1_words, points1, dim, top_k)
            top2 = self._get_top_words_by_dim(comp2_words, points2, dim, top_k)

            results.append({
                'dimension': dim,
                'kl_divergence': kl_divs[dim],
                'cluster1_top': top1,
                'cluster2_top': top2
            })

        return results

    def _get_top_words_by_dim(self, words, vectors, dim, n_words, ascending=False):
        indices = np.argsort(vectors[:, dim])
        if not ascending: indices = indices[::-1]
        return [words[i] for i in indices[:n_words]]

    def print_covariance_analysis(self, results):
        print("\nАнализ различий между кластерами:")
        for res in results:
            print(f"\nИзмерение {res['dimension']} (KL: {res['kl_divergence']:.3f})")
            print(f"Топ кластер 1: {', '.join(res['cluster1_top'][:5])}")
            print(f"Топ кластер 2: {', '.join(res['cluster2_top'][:5])}")

    def compare_clusters_visually(self, cluster_indices, max_points=500, label_top_n=20):
        combined_mask = np.zeros(len(self.reduced_vectors), dtype=bool)
        for idx in cluster_indices:
            combined_mask |= (self.gmm.predict(self.reduced_vectors) == idx)

        points = self.reduced_vectors[combined_mask]
        words = [self.vocab[i] for i in np.where(combined_mask)[0]]

        tsne = TSNE(n_components=2, random_state=42)
        embedded = tsne.fit_transform(points)

        plt.figure(figsize=(20,16))
        texts = []
        for i in range(len(points)):
            plt.scatter(embedded[i,0], embedded[i,1], s=50, alpha=0.7)
            texts.append(plt.text(embedded[i,0], embedded[i,1], words[i], fontsize=8))

        adjust_text(texts)
        plt.title(f"Сравнение кластеров {cluster_indices}")
        plt.axis('off')
        plt.show()

In [ ]:
if __name__ == "__main__":
    analyzer = BertSemanticAnalyzer()

    sentences = analyzer.extract_sentences("annot.opcorpora.xml")
    nouns = analyzer.extract_frequent_nouns(sentences, top_n=10000)

    torch.cuda.empty_cache()
    gc.collect()

    vectors = analyzer.get_contextual_embeddings(
        sentences,
        nouns,
        batch_size=64,
        max_contexts_per_noun=15,
        sample_size=None
    )

    # Анализ
    analyzer.reduce_dimensions(n_components=300, variance_threshold=0.0025)
    analyzer.find_optimal_clusters(max_k=150, min_k=50, step=10)

    # Визуализация и отчет
    analyzer.visualize_clusters(max_points=2000)
    analyzer.print_cluster_words(top_n=15)
    quality_report = analyzer.analyze_cluster_quality()

    # Иерархическая кластеризация с новыми параметрами
    analyzer.hierarchical_clustering(
        max_subclusters=5,
        min_subclusters=3,
        max_points=200,
        min_cluster_size=60
    )

    # Дополнительный анализ
    if len(analyzer.sub_level_gmms) >= 2:
        valid_clusters = list(analyzer.sub_level_gmms.keys())

        if quality_report and 'coherence' in quality_report:
            cluster_coherence = quality_report['coherence']
            filtered_clusters = [(k, cluster_coherence[k]) for k in valid_clusters if k in cluster_coherence]
            sorted_clusters = sorted(filtered_clusters, key=lambda x: x[1])

            if len(sorted_clusters) >= 2:
                cluster1 = sorted_clusters[0][0]
                cluster2 = sorted_clusters[-1][0]
            else:
                cluster1, cluster2 = valid_clusters[:2]
        else:
            cluster1, cluster2 = valid_clusters[:2]

        print("\n" + "="*70)
        print(f"Сравниваем кластеры {cluster1} и {cluster2}")
        if quality_report and 'coherence' in quality_report:
            print(f"Когерентность: {cluster_coherence.get(cluster1, 0):.3f} vs {cluster_coherence.get(cluster2, 0):.3f}")
        print("="*70)

        cov_results = analyzer.analyze_covariance_diff(cluster1, cluster2)
        analyzer.print_covariance_analysis(cov_results)
        analyzer.compare_clusters_visually([cluster1, cluster2])
    else:
        print("\nНедостаточно кластеров для сравнения")